In [1]:
import pandas as pd
import time
import os
from dotenv import load_dotenv
load_dotenv()
#time.sleep(60)

True

In [2]:
#pip install selenium webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
if 1:
    # Reading value
    %store -r lngserieidstart
    print("lngserieidstart = ",lngserieidstart) 
    lngserieidstart -= 1
    print("lngserieidstart = ",lngserieidstart) 


lngserieidstart =  314903
lngserieidstart =  314902


In [4]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [5]:
driver.get("https://www.themoviedb.org/login")
time.sleep(10)


In [6]:
username_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
username_field.clear()
username_field.send_keys(os.getenv("TMDB_LOGIN"))

password_field = driver.find_element(By.NAME, "password")
password_field.clear()
password_field.send_keys(os.getenv("TMDB_PASSWORD"))
password_field.send_keys("\n")


In [7]:
data = pd.read_csv('./data/T_WC_TMDB_SERIE_UPDATE_20260226-fix.csv', sep=';', quotechar='"')
lngserieidstart = 0

In [8]:
data.shape

(909, 8)

In [9]:
def f_tmdbseriesetwikidataid(lngid, strwikidataid):
    driver.get(f"https://www.themoviedb.org/tv/{lngid}/edit?active_nav_item=external_ids")
    intpagefound = True
    try:
        elements = driver.find_elements(By.XPATH, "//h2[text()=\"Oops! We can't find the page you're looking for\"]")
        if len(elements) > 0:
            print("Element 'Page 404' exists on the page.")
            intpagefound = False
        else:
            print("Element 'Page 404' does not exist on the page.")
    except NoSuchElementException:
        print("Element 'Page 404' does not exist on the page.")
    if intpagefound:
        # Define the XPath for the button containing the specific span text
        xpath = "//button[span[contains(@class, 'glyphicons_v2') and contains(@class, 'plus') and contains(@class, 'svg')] and contains(., 'Create Translation')]"
        try:
            # Locate the button using XPath
            button = driver.find_element(By.XPATH, xpath)
            # Click the button
            button.click()
            print("Button 'Create translation' clicked successfully.")
            time.sleep(2)
            driver.get(f"https://www.themoviedb.org/tv/{lngid}/edit?active_nav_item=external_ids")
        except Exception as e:
            print("Button 'Create translation' not found, translation already exists.")
        try:
            wikidata_id_field = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "wikidata_id"))
            )
        except TimeoutException:
            print(f"wikidata_id field not found for serie {lngid}. Skipping.")
            return
        # Is the record locked? 
        css_selector = "span#wikidata_id_status.glyphicons_v2.locked.locked_status"
        elements = driver.find_elements(By.CSS_SELECTOR, css_selector)

        # Check if the element exists by verifying the list is not empty
        if len(elements) > 0:
            # Record is locked
            print("Element 'Locked' exists on the page.")
        else:
            # Record is not locked
            wikidata_id_field.clear()
            wikidata_id_field.send_keys(strwikidataid)

            #save_button = driver.find_element(By.XPATH, "//button[@type='submit']")
            save_button = driver.find_element(By.XPATH, '//input[@value="Save"]')
            save_button.click()

In [10]:
def f_tmdbserieclearwikidataid(lngid):
    driver.get(f"https://www.themoviedb.org/tv/{lngid}/edit?active_nav_item=external_ids")
    wikidata_id_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "wikidata_id"))
    )
    # Is the record locked?
    css_selector = "span#wikidata_id_status.glyphicons_v2.locked.locked_status"
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    if len(elements) > 0:
        print("Element 'Locked' exists on the page. Skipping clear.")
        return

    wikidata_id_field.clear()

    #save_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    save_button = driver.find_element(By.XPATH, '//input[@value="Save"]')
    save_button.click()

In [11]:
lngprocessedcount = 0
for index, row in data.iterrows():
    print(f"Index: {index} ; processed: {lngprocessedcount}")
    if row['ID_SERIE']:
        lngid = row['ID_SERIE']
        if row['ID_WIKIDATA']:
            strwikidataid = row['ID_WIKIDATA']
            strtmdbidtoerase = row['ID_SERIE_ERASE_WIKIDATA_ID']
            if lngid > lngserieidstart:
                # Check if strtmdbidtoerase is nan
                if pd.isna(strtmdbidtoerase):
                    print("strtmdbidtoerase is NaN")
                else:
                    print("strtmdbidtoerase is not NaN")
                    # Convert to int
                    lngtmdbidtoerase = int(strtmdbidtoerase)
                    f_tmdbserieclearwikidataid(lngtmdbidtoerase)
                #print("strtmdbidtoerase",strtmdbidtoerase)
                f_tmdbseriesetwikidataid(lngid,strwikidataid)
                lngprocessedcount += 1
                #f_tmdbserieclearwikidataid(lngid)
                lngserieidstart = lngid
                %store lngserieidstart
                time.sleep(2)
        else:
            print("Colonne ID_WIKIDATA manquante dans le fichier CSV")
    else:
        print("Colonne ID_SERIE manquante dans le fichier CSV")



Index: 0 ; processed: 0
strtmdbidtoerase is NaN
Element 'Page 404' does not exist on the page.
Button 'Create translation' not found, translation already exists.
Element 'Locked' exists on the page.
Stored 'lngserieidstart' (int)
Index: 1 ; processed: 1
strtmdbidtoerase is NaN
Element 'Page 404' does not exist on the page.
Button 'Create translation' not found, translation already exists.
Element 'Locked' exists on the page.
Stored 'lngserieidstart' (int)
Index: 2 ; processed: 2
strtmdbidtoerase is NaN
Element 'Page 404' does not exist on the page.
Button 'Create translation' not found, translation already exists.
Stored 'lngserieidstart' (int)
Index: 3 ; processed: 3
strtmdbidtoerase is NaN
Element 'Page 404' does not exist on the page.
Button 'Create translation' not found, translation already exists.
Stored 'lngserieidstart' (int)
Index: 4 ; processed: 4
strtmdbidtoerase is not NaN
Element 'Locked' exists on the page. Skipping clear.
Element 'Page 404' does not exist on the page.
Bu

In [12]:
if 0:
    # Reading value
    %store -r lngserieidstart
    print("lngserieidstart = ",lngserieidstart) 

In [13]:
print("lngserieidstart = ",lngserieidstart)

lngserieidstart =  315094


In [14]:
# Loop is finished so we display the home page
driver.get("https://www.themoviedb.org/")